# Recitation 4
Topics: Anonymous Functions, Semantics, Scoping

## Anonymous functions

Syntax:
```
Scala:  (param1, param2) => expression
Python: lambda param1, param2: expression
Java:   (param1, param2) -> expression
```

### Exercise: Anonymous func intro
Make anonymous functions that perform the requested operations. (Ok, we're putting them in variables, so they're not quite "anonymous")

**WARNING**: `return` doesn't do what you expect in anonymous functions: https://stackoverflow.com/questions/17754976/scala-return-statements-in-anonymous-functions

In [1]:
// Returns true if input is 1, false otherwise
// BEGIN SOLUTION
val is_one:            (Int) => Boolean = x => x == 1
val is_one_underscore: (Int) => Boolean = _ == 1 // (alternative) Shorthand underscore match syntax
// END SOLUTION
assert(is_one(1))
assert(!is_one(2))

// new: pattern matching
// Returns true if input is 1, false otherwise *using patterrn matching*
// BEGIN SOLUTION
val is_one_pattern:       (Int) => Boolean = x => x match {
    case 1 => true
    case _ => false
}
val is_one_pattern_short: (Int) => Boolean = { // (alternative) Shorthand pattern match syntax `x => x match` is assumed if you have cases
    case 1 => true
    case _ => false
}
// END SOLUTION
assert(is_one_pattern(1))
assert(!is_one_pattern(2))

// new: multi-param
// Returns the addition of the inputs
// BEGIN SOLUTION
val add: (Int, Int) => Int = (x, y) => x + y
// END SOLUTION
assert(add(1, 2) == 3)

// new: return funcs
// Returns a function that adds the numbers to it's input
// BEGIN SOLUTION
val make_adder: Int => Int => Int = x => y => x + y // Also can be thought of as a curried add function
// END SOLUTION
assert(make_adder(1)(2) == 3)

// new: take funcs
// Takes a function and applies it to 3
// BEGIN SOLUTION
val call_on_3: (Int => Int) => Int = f => f(3)
// END SOLUTION
assert(call_on_3((x) => x + 5) == 8)

// new: statements
// Any anonymous function with a loop in it
// BEGIN SOLUTION
val func_with_loop: (Int) => Unit = x => {
    for (i <- 0 until x) {
        println("hi"+ i.toString)
    }
}
// END SOLUTION
func_with_loop(10)

hi0
hi1
hi2
hi3
hi4
hi5
hi6
hi7
hi8
hi9


is_one: Int => Boolean = <function1>
is_one_underscore: Int => Boolean = <function1>
is_one_pattern: Int => Boolean = <function1>
is_one_pattern_short: Int => Boolean = <function1>
add: (Int, Int) => Int = <function2>
make_adder: Int => Int => Int = <function1>
call_on_3: Int => Int => Int = <function1>
func_with_loop: Int => Unit = <function1>

### Exercise: Mini interpreter
Implement the function below to apply the operations, from left to right of the list to the input by passing an anonymous function to fold

In [2]:
sealed trait Operation
case class Plus(n: Double) extends Operation
case class Minus(n: Double) extends Operation
case class Pow(n: Double) extends Operation
case class NestedOps(ops: List[Operation]) extends Operation
case object Sin extends Operation

// Version 1: Loop with acc
def perform_ops_loop(init: Double, steps: List[Operation]): Double = {
    var v = init
    for (ops <- steps) {
        v = { ops match {
            case Plus(n) => v + n
            case Minus(n) => v - n
            case Pow(n) => math.pow(v, n)
            case NestedOps(ops) => perform_ops_loop(v, ops)
            case Sin => math.sin(v)
            }
        }
    }
    return v
}

// Version 2: Using recursion
def perform_ops_rec(v: Double, steps: List[Operation]): Double = {
    if (steps.length == 0) v
    else {
        val o = steps.head
        val rest = steps.tail
        o match {
            case Plus(n) => perform_ops_rec(v + n, rest)
            case Minus(n) => perform_ops_rec(v - n, rest)
            case Pow(n) => perform_ops_rec(math.pow(v, n), rest)
            case NestedOps(ops) => perform_ops_rec( perform_ops_rec(v, ops), rest)
            case Sin => perform_ops_rec(math.sin(v), rest)
        }
    }
}
// Version 3: using foldLeft
def perform_ops(init: Double, steps: List[Operation]): Double =
    steps.foldLeft(init)(
        (acc, op) => op match {
            case Plus(n) => acc + n
            case Minus(n) => acc - n
            case Pow(n) => Math.pow(acc, n)
            case NestedOps(ops) => perform_ops(acc, ops)
            case Sin => Math.sin(acc)
        }
    )

assert(perform_ops(5, List(Plus(5), Pow(2), NestedOps(List(Minus(3))))) == 97)

defined trait Operation
defined class Plus
defined class Minus
defined class Pow
defined class NestedOps
defined object Sin
defined function perform_ops_loop
defined function perform_ops_rec
defined function perform_ops

### Exercise: Operartional semantics
Write inference rules to define `Plus` and `NestedOps` (on paper).

#### Answer:

$$
\newcommand{\cons}[2]{\texttt{#1(}#2\texttt{)}}
\begin{array}{c}
init + n = n' \\
perform\_ops(n', rest\_ops) = n'' \\
\hline
perform\_ops(init,\ \cons{Plus}{n} :: rest\_ops) = n'' \\
\end{array} \text{(plus)}
$$



$$
\begin{array}{c}
perform\_ops(init,\ ops) = n'  \\
perform\_ops(n', rest\_ops) = n'' \\
\hline
perform\_ops(init,\ \cons{NestedOps}{ops}) :: rest\_ops = n'' \\
\end{array} \text{(nested ops)}
$$

### Exercise: Sort
Don't worry, you don't have to write quick-sort. Use Scala's built in `sortWith` to sort food in the requested ways with anonymous functions.

```scala
// Method of list class
def sortWith(lt: (A, A) ⇒ Boolean): List[A]
```

In [3]:
case class Food(val calories: Double, val price: Double) // Like a sealed trait with fields

val donut = Food(300, 1.50)
val goat_cheese_spinach_kale_and_soy_infused_kombucha = Food(-50, 1000)
val buffalo_chicken_pizza = Food(500, 20)

val menu = List(donut, goat_cheese_spinach_kale_and_soy_infused_kombucha, buffalo_chicken_pizza)

// BEGIN SOLUTION
val sorted_by_calories = menu.sortWith((f1, f2) => f1.calories < f2.calories)
val sorted_by_price = menu.sortWith((f1, f2) => f1.price < f2.price)
// END SOLUTION

assert(sorted_by_calories == List(goat_cheese_spinach_kale_and_soy_infused_kombucha, donut, buffalo_chicken_pizza))
assert(sorted_by_price == List(donut, buffalo_chicken_pizza, goat_cheese_spinach_kale_and_soy_infused_kombucha))

defined class Food
donut: Food = Food(300.0, 1.5)
goat_cheese_spinach_kale_and_soy_infused_kombucha: Food = Food(-50.0, 1000.0)
buffalo_chicken_pizza: Food = Food(500.0, 20.0)
menu: List[Food] = List(Food(300.0, 1.5), Food(-50.0, 1000.0), Food(500.0, 20.0))
sorted_by_calories: List[Food] = List(Food(-50.0, 1000.0), Food(300.0, 1.5), Food(500.0, 20.0))
sorted_by_price: List[Food] = List(Food(300.0, 1.5), Food(500.0, 20.0), Food(-50.0, 1000.0))

## Scoping

###  Exercise: Lettuce concrete syntax scoping
For each variable use below, match each use with its declaration.
Numbers are used to label uses, capital leters are used to label declarations.

```
let a = 5 in
    A

    let b = a in
        B   1

        let a = b in a + b
            C   2    3   4
```


| Use | Declaration|
|-----|------------|
|1| A|
|2| B|
|3| C|
|4| B|

```
let a = function(a)
    A            B
    
            let a = a in
                C   1
                
                a(5) + 4
                2

    in

        function(b)
                 D

            let a = a(let a = 5 in a) in
                E   3     F        4

                a(a(b))
                5 6 7
```

| Use | Declaration|
|-----|------------|
|1| B|
|2| C|
|3| A|
|4| F|
|5| E|
|6| E|
|7| D|

###  Exercise: Lettuce abstract syntax scoping
For each variable use below, match each use with its declaration.
Numbers are used to label uses, capital leters are used to label declarations.

```
Let("x",
     A

    Let("f",
         B
         
        FunDef("y"
                C
        
               Plus(Ident("x"), Ident("y")))
                           1          2
               
        FunCall(Ident("f"), Const(3))),
                       3
                       
    Let("y",
         D
    
        Ident("x"),
               4

        FunCall(Ident("f"), Ident("y"))))
                       5           6
```

| Use | Declaration|
|-----|------------|
|1| A|
|2| C|
|3| B|
|4| A|
|5| Error (out of scope, not defined)|
|6| D|

###  Exercise: Scala concrete syntax scoping
For each variable use below, match each use with its declaration.
Numbers are used to label uses, capital leters are used to label declarations.

In [4]:
   val x = 5
//     A

   val y = x + 3
//     B   1

   val f2: (Int => Int) => Int = (y) => {
//     C                          D

       y(x)
//     2 3

   }

   val f3: ((Int => Int) => Int) => Int =
//     E

       (f3) => f3((f3) => f3)
//      F      4   G      5

   f3(f2)
// 6  7

x: Int = 5
y: Int = 8
f2: Int => Int => Int = <function1>
f3: Int => Int => Int => Int = <function1>
res3_4: Int = 5

| Use | Declaration|
|-----|------------|
|1| A|
|2| D|
|3| A|
|4| F|
|5| G|
|6| E|
|7| C|

In [5]:
   val x = 5
//     A
   
   x match {
// 1
       
       case x if x == x => {
//          B    2    3
       
           val x = 4
//             C

           x
//         4
           
       }
       
       case y if x == y =>
//          D    5    6
       
           ((x: Int) => y + x)(x)
//           E          7   8  9

       // Extra credit...
       case `x` => x match {
//           10    11

           case x @ `x` => x
//              F    12    13

       }
           
   }

x: Int = 5
res4_1: Int = 4

| Use | Declaration|
|-----|------------|
|1| A|
|2| B|
|3| B|
|4| C|
|5| A|
|6| D|
|7| D|
|8| E|
|9| A|
|10| A|
|11| A|
|12| A|
|13| F|

### End
* Project
    * Running
    * Testing
    * Zipping
* Assignment 4
* Assignment 5